In [46]:
import numpy as np
from prettytable import PrettyTable
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm
from scipy.special import erf

Варіант 22

In [2]:
# Теоретичний розподіл: РОЗПОДІЛ ПУАССОНА

data = np.array([12, 15, 7, 19, 15, 27, 22, 4,
                 11, 19, 15, 7, 1, 12, 22, 4,
                 11, 15, 19, 12, 7, 27, 15, 19,
                 11, 22, 12, 19, 15, 12, 15, 31])
y = 0.999
sigma = 7
a = 0.05

In [3]:
var_arr = np.sort(data)
values, counts = np.unique(var_arr, return_counts=True)
n = np.size(data)

table = PrettyTable()
table.field_names = ["Унікальне значення", "Кількість"]
for i in range(np.size(values)):
    table.add_row([values[i], counts[i]], divider=True)
table

Унікальне значення,Кількість
1,1
4,2
7,3
11,3
12,5
15,7
19,5
22,3
27,2
31,1


Знайдіть точкові незміщені оцінки середнього значення та середнього квадратичного відхилення

In [4]:
x_b = round(np.sum(values * counts) / n, 6)
print(x_b)

np.float64(14.8125)

In [5]:
D_b = round(np.sum(pow(values, 2) * counts) / n, 6) - pow(x_b, 2)
S2 = D_b*n/(n-1)
if n>=30:
    sigma_b = round(np.sqrt(D_b), 6)
else:
    sigma_b = round(np.sqrt(S2), 6)
print(sigma_b)

np.float64(6.830252)

Припускаючи нормальний розподіл випадкової величини, знайдіть з надійністю у довірчий інтервал для математичного сподівання при відомому σ 

In [60]:
# F_x = y/2 = 0.4995
x_res = 3.4

top_limit = x_b - x_res*sigma/np.sqrt(n)
bottom_limit = x_b + x_res*sigma/np.sqrt(n)

print(x_res)
print(f'{round(top_limit, 6)} < M(x) < {round(bottom_limit, 6)}')

3.4
10.605215 < M(x) < 19.019785


In [59]:
x_res1 = norm.ppf(y)

top_limit = x_b - x_res1*sigma/np.sqrt(n)
bottom_limit = x_b + x_res1*sigma/np.sqrt(n)

print(round(x_res1, 2))
print(f'{round(top_limit, 6)} < M(x) < {round(bottom_limit, 6)}')

3.09
10.988533 < M(x) < 18.636467


https://stackoverflow.com/questions/56016484/how-to-calculate-laplace-function-in-python-3

In [63]:
F_x = lambda x: erf(x/2**0.5)/2
x_res2 = F_x(y)

top_limit = x_b - x_res2*sigma/np.sqrt(n)
bottom_limit = x_b + x_res2*sigma/np.sqrt(n)

print(round(x_res2, 2))
print(f'{round(top_limit, 6)} < M(x) < {round(bottom_limit, 6)}')

0.34
14.390407 < M(x) < 15.234593


Перевірте за допомогою критерію Пірсона при рівні значущості α гіпотезу про відповідний теоретичний розподіл заданої випадкової величини